In [ ]:
# =========================================
# SISTEMA DE RECOMENDAÇÃO DE PRATOS BRASILEIROS (TF-IDF)
# Inclui pausa após a explicação matemática
# =========================================

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# =========================================
# BASE DE DADOS DOS PRATOS CASEIROS
# =========================================

data = {
    'Prato': [
        'Feijoada',
        'Bife a cavalo',
        'Strogonoff de frango',
        'Lasanha à bolonhesa',
        'Moqueca de peixe',
        'Escondidinho de carne seca',
        'Feijão tropeiro',
        'Galinhada',
        'Bobó de camarão',
        'Carne de panela com batata',
        'Macarronada com almôndegas',
        'Peixe frito com arroz e salada',
        'Virado à paulista',
        'Picadinho de carne'
    ],
    'Ingredientes': [
        'feijão preto, carne seca, costelinha, linguiça, arroz, farofa, couve',
        'arroz, feijão, bife, ovo, batata frita, salada',
        'frango, creme de leite, ketchup, batata palha, arroz',
        'massa, carne moída, molho de tomate, queijo, presunto',
        'peixe, leite de coco, pimentão, tomate, coentro, cebola',
        'carne seca, mandioca, queijo, manteiga',
        'feijão, farinha de mandioca, linguiça, torresmo, ovo',
        'arroz, frango, açafrão, cenoura, pimentão, alho',
        'camarão, mandioca, leite de coco, azeite de dendê, cebola, coentro',
        'carne, batata, tomate, cebola, alho',
        'macarrão, almôndegas, molho de tomate, queijo',
        'peixe, arroz, salada, tomate, alface, limão',
        'arroz, tutu de feijão, bisteca, ovo, banana frita',
        'carne, cebola, molho de soja, alho, arroz, farofa'
    ]
}

df = pd.DataFrame(data)

# =========================================
# VETORIZAÇÃO DOS INGREDIENTES (TF-IDF)
# =========================================

vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b')
tfidf_matrix = vectorizer.fit_transform(df['Ingredientes'])

# =========================================
# CÁLCULO DA SIMILARIDADE ENTRE OS PRATOS
# =========================================

similarity_matrix = cosine_similarity(tfidf_matrix)

# =========================================
# CLASSIFICAÇÃO DA SIMILARIDADE
# =========================================

def classificar_similaridade(valor):
    if valor <= 0.33:
        return "Baixa"
    elif valor <= 0.66:
        return "Média"
    else:
        return "Alta"

# =========================================
# EXPLICAÇÃO MATEMÁTICA
# =========================================

def mostrar_explicacao():
    print("\n📘 Explicação Matemática:")
    print("""
A similaridade é calculada usando o **Cosseno do Ângulo** entre os vetores TF-IDF dos pratos.

Cada prato é representado como um vetor numérico (V₁, V₂, ... Vₙ), onde cada posição indica a importância
de um ingrediente no prato (TF-IDF).

A fórmula usada é:

            Similaridade = (A · B) / (||A|| * ||B||)

onde:
 - (A · B) é o produto escalar entre os vetores A e B
 - ||A|| e ||B|| são as normas (comprimentos) dos vetores

O valor da similaridade varia de:
 - 1 → vetores idênticos (ângulo 0°, alta similaridade)
 - 0 → vetores ortogonais (ângulo 90°, nenhuma similaridade)

O ângulo θ entre os vetores é calculado por:
            θ = arccos(Similaridade)
""")
    print("---------------------------------------------")
    input("Pressione ENTER para voltar ao menu...")

# =========================================
# FUNÇÃO DE RECOMENDAÇÃO
# =========================================

def recomendar_prato(indice_prato, top_n=3):
    nome_prato = df.loc[indice_prato, 'Prato']
    idx = indice_prato
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    print(f"\n🍽️ Recomendações baseadas em '{nome_prato}':\n")

    for i, (ind, score) in enumerate(scores[1:top_n+1], start=1):
        prato_recomendado = df['Prato'][ind]
        nivel = classificar_similaridade(score)
        angulo = np.degrees(np.arccos(score))  # ângulo em graus
        print(f"{i}. {prato_recomendado}  →  Similaridade: {score:.2f}  ({nivel})  →  Ângulo: {angulo:.1f}°")

    print("\n💡 Se quiser ver a explicação matemática, digite 'Explicação'.")
    print("Ou pressione ENTER para continuar.\n")

    resposta = input("👉 Sua escolha: ").strip().lower()
    if resposta == 'explicação':
        mostrar_explicacao()
    else:
        print("---------------------------------------------")

# =========================================
# LOOP INTERATIVO
# =========================================

def mostrar_menu():
    print("\n📋 PRATOS DISPONÍVEIS:")
    for i, p in enumerate(df['Prato'], start=1):
        print(f"{i} → {p}")
    print("\n💡 Digite o número do prato que deseja ver recomendações.")
    print("👉 Digite 'sair' para encerrar o programa.\n")

# Loop principal
while True:
    mostrar_menu()
    escolha = input("🔎 Escolha um número: ").strip()

    if escolha.lower() == 'sair':
        print("\n👋 Encerrando o sistema de recomendação. Até a próxima!")
        break

    if not escolha.isdigit():
        print("⚠️ Por favor, digite um número válido.")
        continue

    escolha = int(escolha)
    if escolha < 1 or escolha > len(df):
        print("⚠️ Número inválido. Escolha um dos números listados.")
        continue

    recomendar_prato(escolha - 1)
